In [1]:
import agentpy as ap  # Library for creating agents
import numpy as np  # NumPy library for numerical operations
import matplotlib.pyplot as plt  # Library for plotting
import seaborn as sns  # Library for statistical data visualization
from random import randint  # For generating random numbers
import IPython  # For displaying videos in the notebook
from matplotlib.animation import FuncAnimation  # For creating animations (used by agentpy
from queue import PriorityQueue  # For creating priority queues

In [5]:
class PathfindingAgent(ap.Agent):
    def setup(self):
        self.pathfinding_id = 2 # Pathfinding agent ID
        self.destination = None # Destination agent
        self.path = None # Path to the destination (list of coordinates)
        self.position = None # Current position
        self.found_path = False # Flag to indicate if a path has been found
        self.is_obstacle = np.nan # Flag to indicate if the agent is an obstacle
        self.path_id = None # ID shared between agent and its destination to match them
        self.patience = 3 # Number of steps to wait for another agent to move
        self.time_waited = 0 # Number of steps the agent has waited

    def action(self):
        if not self.found_path: # If a path has not been found
            #grid_array = self.model.grid.attr_grid('is_obstacle')  # Use 'is_obstacle'
            #grid_array = self.prepare_grid(grid_array, self.get_position(), self.destination)
            #print("Grid array og:", grid_array)
            #self.path = self.find_path(grid_array)
            #self.path.append(self.destination)  # Add the destination to the path
            self.prepare_grid()
            
            if self.path:
                self.found_path = True
                print("Path found:", self.path)
                # Optionally, remove the current position from the path if immediate move is desired
                if self.path[0] == self.position:
                    self.path.pop(0)
            else:
                print("No path found")
                return  # Exit if no path is found
        

        if self.path: # If a path is available
            # Determine the next step to take
            next_step = self.path.pop(0)  # Get the next coordinate in the path

            # Calculate the relative move from the current position
            current_position = self.get_position()
            move_direction = (next_step[0] - current_position[0], next_step[1] - current_position[1])

            # Check if theres an agent in the position to move to
            if len(self.model.grid.agents[(current_position[0] + move_direction[0]), (current_position[1] + move_direction[1])]) > 0:
                print("Agent at the next position, checking if it's an obstacle")

                # Check if the agent at the next position is another pathfinding agent
                agents_at_next_pos = self.model.grid.agents[(current_position[0] + move_direction[0]), (current_position[1] + move_direction[1])]
                is_pathfinding_agent = False
                for agent in agents_at_next_pos:
                    if agent.pathfinding_id == 2:
                        is_pathfinding_agent = True
                        break

                if is_pathfinding_agent:
                    print("Agent at the next position is another pathfinding agent")
                    # Add next step back to the path and wait for agent to move
                    self.path.insert(0, next_step)
                    # Reduce patience by 1, 50% chance of reducing
                    if randint(0, 1) == 0:
                        self.patience -= 1
                    # Check if the agent has waited long enough
                    if self.patience == 0:
                        print("Agent waited long enough, recalculating path")
                        #set the agents in front of it as obstacles temporarily
                        agents_to_modify = []
                        for agent in agents_at_next_pos:
                            if agent.is_obstacle != 3:
                                agent.is_obstacle = 3
                                agents_to_modify.append(agent)



                        # Recalculate grid
                        grid_array = self.model.grid.attr_grid('is_obstacle')
                        grid_array = self.prepare_grid(grid_array, self.get_position(), self.destination.get_position())
                        print("Grid array recalc:", grid_array)
                        # Reset the agents in front of it
                        for agent in agents_to_modify:
                            agent.is_obstacle = np.nan

                        # Reset the path
                        self.path = None
                        self.found_path = False


                        # Recalculate path
                        self.path = self.find_path(grid_array)
                        self.path.append(self.destination.get_position())
                        if self.path:
                            self.found_path = True
                            print("Path found:", self.path)
                            # Optionally, remove the current position from the path if immediate move is desired

                        else:
                            print("No path found")
                            return

                        # Reset patience

                        self.patience = 3
                    self.time_waited += 1
                    return

            # Move the agent by the calculated direction
            self.model.grid.move_by(self, move_direction)
            self.time_waited = 0

            # Check if the agent has reached the destination or time to wait has been exceeded 5 steos
            if self.get_position() == self.destination.get_position() or self.time_waited > 5:
                print("Agent reached the destination")
                self.model.grid.remove_agents(self)
                self.model.remaining_agents -= 1
                # Remove the destination from the grid
                #agents_to_remove = [agent for agent in self.model.grid.agents[self.destination] if agent != self]
                self.model.grid.remove_agents(self.destination)
                
                # Now remove the collected agents from the grid
                #for agent in agents_to_remove:
                #   self.model.grid.remove_agents(agent)
       # else:
        #    print("Agent at destination or no path available")
                    
    def prepare_grid(self):
        grid_array = self.model.grid.attr_grid('is_obstacle')  # Use 'is_obstacle'
        grid_array = self.prepare_grid(grid_array, self.get_position(), self.destination.get_position()) # Prepare the grid
        self.path = self.find_path(grid_array)  # Find the path

    def prepare_grid(self, grid_array, start_pos, end_pos):
        # Prepares the grid by setting the start and end positions
        # Convert to a writable array first if necessary
        grid_copy = np.array(grid_array, dtype=np.float64)
        grid_copy[start_pos] = 2  # Mark the start position
        grid_copy[end_pos] = 1    # Mark the destination position
        return grid_copy

    def find_path(self, grid_array):
    
        def h(p1, p2):
            return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

        def reconstruct_path(came_from, current):
            path = []
            while current in came_from:
                current = came_from[current]
                path.append(current)
            return path[::-1]

        # Convert grid_array to treat NaNs as 0 for open path, and obstacles (3) as inf
        grid_array = np.where(np.isnan(grid_array), 0, grid_array)
        grid_array = np.where(grid_array == 3, float('inf'), grid_array)

        start = None
        end = None
        for i in range(grid_array.shape[0]):
            for j in range(grid_array.shape[1]):
                if grid_array[i, j] == 2:
                    start = (i, j)
                elif grid_array[i, j] == 1:
                    end = (i, j)
        
        if not start or not end:
            return []  # If there's no start or end, return an empty path.

        open_set = PriorityQueue()
        open_set.put((0, start))
        came_from = {}
        g_score = {spot: float("inf") for spot in np.ndindex(grid_array.shape)}
        g_score[start] = 0
        f_score = {spot: float("inf") for spot in np.ndindex(grid_array.shape)}
        f_score[start] = h(start, end)

        while not open_set.empty():
            current = open_set.get()[1]

            if current == end:
                return reconstruct_path(came_from, end)

            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:  # NSEW movements
                neighbor = (current[0] + dx, current[1] + dy)
                if 0 <= neighbor[0] < grid_array.shape[0] and 0 <= neighbor[1] < grid_array.shape[1]:
                    # Check if the neighbor is an obstacle
                    if grid_array[neighbor[0], neighbor[1]] == float('inf'):
                        continue  # Skip this neighbor
                    
                    temp_g_score = g_score[current] + 1

                    if temp_g_score < g_score[neighbor]:
                        came_from[neighbor] = current
                        g_score[neighbor] = temp_g_score
                        f_score[neighbor] = temp_g_score + h(neighbor, end)
                        if neighbor not in {item[1] for item in open_set.queue}:  # Check if not in open_set
                            open_set.put((f_score[neighbor], neighbor))

        print("No path found")
        return []  # Return an empty path if there's no path to the end.

    def get_position(self):
        return self.model.grid.positions[self]

NameError: name 'ap' is not defined

In [ ]:
class DestinationAgent(ap.Agent):
    def setup(self):
        self.pathfinding_id = 1
        self.is_obstacle = np.nan
        self.path_id=None # ID shared between agent and its destination to match them

    def get_position(self):
        return self.model.grid.positions[self]

In [ ]:
class ObstacleAgent(ap.Agent):
    def setup(self):
        self.pathfinding_id = 3
        self.is_obstacle = 3 

In [ ]:
class PathfindingModel(ap.Model):
    def setup(self):
        # Create agents
        self.pathfinding_agents = ap.AgentList(self, self.p.n_agents, PathfindingAgent)
        self.destination_agents = ap.AgentList(self, self.p.n_agents, DestinationAgent)
        self.obstacle_agents = ap.AgentList(self, self.p.n_obstacles, ObstacleAgent)

        # Create grid
        self.grid = ap.Grid(self, (self.p.dimensions, self.p.dimensions), track_empty=True, check_border=True)

        # Add agents to grid
        self.grid.add_agents(self.pathfinding_agents, random=True, empty=True)
        self.grid.add_agents(self.destination_agents, random=True, empty=True)
        self.grid.add_agents(self.obstacle_agents, random=True, empty=True)

        for i in range(len(self.pathfinding_agents)):
            agent = self.pathfinding_agents[i]
            destination = self.destination_agents[i]
            agent.position = agent.get_position()
            print("agent position: " + str(agent.position))
            agent.destination  = destination
            print("destination position: " + str(agent.destination.get_position()))

        self.remaining_agents = self.p.n_agents



    def step(self):
        for agent in self.pathfinding_agents:
            agent.action()

    def update(self):
        if self.remaining_agents == 0:
            self.stop()



    def end(self):
        print("\n")
        print(self.grid.attr_grid('pathfinding_id'))

In [ ]:
parameters={
    'steps': 30,
    'dimensions': 50,
    'n_agents': 20,
    'n_obstacles': 200
}
print("MODEL RESULTS \n")
model = PathfindingModel(parameters)

results = model.run()

print("attr-grid \n")
print(model.grid.attr_grid('pathfinding_id'))




MODEL RESULTS 

agent position: (25, 3)
destination position: (13, 12)
agent position: (42, 2)
destination position: (48, 42)
agent position: (1, 29)
destination position: (12, 28)
agent position: (22, 23)
destination position: (42, 12)
agent position: (47, 25)
destination position: (21, 37)
agent position: (37, 18)
destination position: (19, 38)
agent position: (8, 5)
destination position: (34, 30)
agent position: (4, 30)
destination position: (11, 34)
agent position: (20, 27)
destination position: (18, 9)
agent position: (39, 12)
destination position: (11, 21)
agent position: (24, 37)
destination position: (9, 27)
agent position: (13, 43)
destination position: (19, 42)
agent position: (31, 29)
destination position: (41, 9)
agent position: (39, 27)
destination position: (16, 8)
agent position: (25, 36)
destination position: (1, 6)
agent position: (26, 36)
destination position: (45, 16)
agent position: (25, 32)
destination position: (43, 13)
agent position: (0, 16)
destination position

In [ ]:
def animation_plot(model,ax):
    attr_grid = model.grid.attr_grid('pathfinding_id')

    color_dict = {1: 'green', 2: 'blue', 3: 'red', None: 'white'}

    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)

    ax.set_title("Pathfinding\n"
                "Remaining agents: " + str(model.remaining_agents)+
                "Steps: " + str(model.t))

fig, ax = plt.subplots()

model2 = PathfindingModel(parameters)

animation = ap.animate(model2, fig, ax, animation_plot)

agent position: (4, 39)
destination position: (24, 34)
agent position: (31, 3)
destination position: (24, 47)
agent position: (44, 25)
destination position: (2, 27)
agent position: (11, 22)
destination position: (31, 24)
agent position: (27, 3)
destination position: (41, 26)
agent position: (37, 13)
destination position: (12, 39)
agent position: (4, 16)
destination position: (32, 2)
agent position: (38, 36)
destination position: (4, 29)
agent position: (13, 47)
destination position: (7, 40)
agent position: (20, 28)
destination position: (38, 28)
agent position: (6, 27)
destination position: (26, 5)
agent position: (37, 31)
destination position: (42, 21)
agent position: (29, 1)
destination position: (5, 15)
agent position: (24, 44)
destination position: (5, 46)
agent position: (34, 26)
destination position: (33, 42)
agent position: (33, 38)
destination position: (0, 9)
agent position: (3, 10)
destination position: (13, 38)
agent position: (8, 0)
destination position: (13, 4)
agent posit

In [ ]:
IPython.display.HTML(animation.to_jshtml(fps=5))


Grid array og: [[ 3. nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan  3. nan ... nan  3. nan]
 ...
 [nan nan nan ... nan  3. nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
Path found: [(4, 39), (4, 38), (4, 37), (4, 36), (4, 35), (4, 34), (5, 34), (6, 34), (7, 34), (8, 34), (9, 34), (10, 34), (11, 34), (12, 34), (13, 34), (14, 34), (15, 34), (16, 34), (17, 34), (18, 34), (19, 34), (20, 34), (21, 34), (22, 34), (23, 34), (24, 34)]
Grid array og: [[ 3. nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan  3. nan ... nan  3. nan]
 ...
 [nan nan nan ... nan  3. nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
Path found: [(31, 3), (31, 4), (31, 5), (31, 6), (31, 7), (31, 8), (31, 9), (31, 10), (31, 11), (31, 12), (31, 13), (31, 14), (30, 14), (29, 14), (29, 15), (28, 15), (28, 16), (28, 17), (28, 18), (28, 19), (27, 19), (27, 20), (26, 20), (25, 20), (24, 20), (24, 21), (24, 22), (24, 23), (24, 24), (24, 25), (24, 26), (24, 27)